In [ ]:
# Imports
import IPython, random, math, time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### The goal of this particle simulation is to have particles that can move and interact, as well as having charges, 
### and display that all by console or MAYBE use numpy to make an image and animate that

def exclude(item, arr, index = False):
  '''Excludes an item from the array'''
  copy = arr.copy()
  if index:
    copy.pop(index)
  else:
    copy.pop(copy.index(item))
  return copy
  

class Particle:
  def __init__(self, env = None, icon = ".", size = 1, mass = 1, position = np.zeros((2,)), velocity = np.zeros((2,)), charge = 0, friction = 0.99):
    '''This object will be used to represent a particle:

    icon - An icon that will represent the particle when printed out to the console
    position - It's position in 2-D space
    size - It's size in 2-D (2-D Vector)
    velocity - It's velocity (2-D Vector)
    charge - The sign of its charge as well as its strength
    friction - The rate at which the velocity will naturally decay
    
    '''
    if type(position) != np.ndarray:
      position = np.array(position, dtype= np.float32)
    if type(velocity) != np.ndarray:
      velocity = np.array(velocity, dtype = np.float32)
    
    self.env = env
    self.icon = icon
    self.size = size
    self.mass = mass
    self.position = position
    self.velocity = velocity
    self.charge = charge
    self.friction = friction

    self.coliding_with_another_particle = False

    self.colider_size = np.array([self.size/2, self.size/2], dtype = np.float32)
  
  def set_env(self, env):
    '''Set this particle's env'''
    self.env = env

  def coliders(self):
    '''Finds any and all particles colliding with itself'''
    particles_coliding = self.check_for_particles_coliding()
    if len(particles_coliding) > 0:
      # For debugging purposes, can be deleted later
      self.coliding_with_another_particle = True
      
      for particle in particles_coliding:
        self.hit(particle)
        self.push(particle)

    else:
      self.coliding_with_another_particle = False

  def check_if_going_to_collide_with_any_particles(self):
    '''Interpolates the velocity using the size of the smallest particle and with that finds if any particles are in its path'''
    arr = []

    for i in range(0, self.velocity, self.env.smallest_particle_size):
      for particle in self.other_particles:
        if all(np.abs(particle.position - self.position) < self.colider_size):
          arr.append(particle)

    return arr

  def check_for_particles_coliding(self):
    ''' Checks for all particles coliding with itself'''
    arr = []
    for particle in self.other_particles:
      if all(np.abs(particle.position - self.position) < self.colider_size):
        arr.append(particle)

    return arr

  def check_for_particles_coliding_for_position(self, position):
    '''checks to see if particles are coliding in a position in the environment'''
    arr = []
    for particle in self.other_particles:
      if all(np.abs(particle.position - position) < self.colider_size):
        arr.append(particle)

    return arr

  def apply_forces(self):
    ''' Changes the velocity based on forces like electromagnetic, and if it gets hit by any other particle'''
    self.velocity = ((self.velocity * (self.friction * self.env.friction))) + (self.get_attractive_forces_from_other_particles() * self.env.time_interval * (1/self.mass))

  def set_personal_global_particle_data(self):
    ''' This is to speed up calculations for the charges so we don't have to make a new array each time'''
    self.global_other_particle_data = [np.array([particle.position for particle in exclude(self,self.env.particles)], dtype = np.float16), np.array([particle.charge for particle in exclude(self,self.env.particles)], dtype = np.float16)]

  def set_other_particles(self):
    ''' This is to speed up calculations and to avoid any inf or -inf's in the forces'''
    self.other_particles = exclude(self, self.env.particles)

  def move(self):
    ''' Updates the position based on the velocity '''
    self.position += self.velocity * self.env.time_interval
    self.coliders()

  def get_position(self):
    return self.position.copy()
  
  def get_velocity(self):
    return self.velocity.copy()
  
  def set_friction(self, friction):
    self.friction = friction

  def reset_position(self, n):
    self.position[n] = self.env.size[n] // 2

  def reset_velocity(self, n):
    self.velocity[n] = 0

  def flip_direction(self, dir):
    self.velocity *= np.array(dir)

  def step(self):
    ''' Moves itself and checks if it is out of bounds or if an error occured'''
    self.move()

    # If the particle gets out of bounds of the environment box then the particle will bounce back and its position will be changed so it doesn't go off screen
    if self.position[0] > self.env.size[0]-1:
      self.velocity *= np.array([-self.env.friction,self.env.friction])
      self.position[0] = self.env.size[0]-1
    elif self.position[0] < 0:
      self.velocity *= np.array([-self.env.friction,self.env.friction])
      self.position[0] = 0
    elif self.position[1] > self.env.size[1]-1:
      self.velocity *= np.array([self.env.friction,-self.env.friction])
      self.position[1] = self.env.size[1]-1
    elif self.position[1] < 0:
      self.velocity *= np.array([self.env.friction,-self.env.friction])
      self.position[1] = 0
    
    if np.isnan(self.position[0]) or np.isinf(self.position[0]):
      self.position[0] = random.random() * self.env.size[0]
    if np.isnan(self.position[1]) or np.isinf(self.position[1]):
      self.position[1] = random.random() * self.env.size[1]
    if np.isnan(self.velocity[0]) or np.isinf(self.velocity[0]):
      self.velocity[0] = 0
    if np.isnan(self.velocity[1]) or np.isinf(self.velocity[1]):
      self.velocity[1] = 0
      
        
  def hit(self, particle):
    '''Hits another particle based on self.mass and other.mass and self.velocity and friction'''
    energy_exchanged = ((self.mass / particle.mass * self.velocity)) * self.friction * particle.friction
    particle.velocity += energy_exchanged
    self.velocity -= (energy_exchanged) 
  
  def push(self, particle):
    ''' If there is a situation where the two particles are overlapping then this function will be used to push the two particles'''
    distance_between = np.abs(particle.position - self.position)
    total_size = (self.size + particle.size)
    # Pushes the two particles by half the size
    if distance_between[0] < total_size:
      if self.position[0] < particle.position[0]:
        self.position[0] -= distance_between[0] * 0.5
        particle.position[0] += distance_between[0] * 0.5
      else:
        self.position[0] += distance_between[0] * 0.5
        particle.position[0] -= distance_between[0] * 0.5
    if distance_between[1] < total_size:
      if self.position[1] < particle.position[1]:
        self.position[1] -= distance_between[1] * 0.5
        particle.position[1] += distance_between[1] * 0.5
      else:
        self.position[1] += distance_between[1] * 0.5 
        particle.position[1] -= distance_between[1] * 0.5

  def get_attractive_forces_from_other_particles(self):
    ''' Get the attractive forces from other particles by using the inverse square of the distance and each particle's charge''' 
    if self.charge != 0:
      distance_between = self.global_other_particle_data[0] - self.position
      divided_by_max = distance_between / np.expand_dims(np.sum(np.abs(distance_between),1),1)
      charges = (divided_by_max * np.expand_dims(self.global_other_particle_data[1],1)) * self.charge * -1 * (np.expand_dims(np.sum(distance_between,1),1) ** (-2))
      return np.sum(charges * self.env.coulomb_constant,0)
    return 0

  def info(self):
    ''' Returns a string with some information for debugging purposes'''
    return f"Particle #{self.env.particles.index(self)} | {self.icon}:\n\tposition: [{self.position[0]}, {self.position[1]}]\tvelocity: [{self.velocity[0]}, {self.velocity[1]}]\n\tColiding with another particle: {self.coliding_with_another_particle}"
  
  def copy(self):
    '''Returns a copy of a given particle (this was used for testing two different environments to see butterfly effect)'''
    return Particle(env = None, icon = self.icon, size = self.size, mass = self.mass, position = self.get_position(), velocity = self.get_velocity(), charge = self.charge, friction = self.friction)
    
class Environment:
  def __init__(self, particles, size = (98, 23), time_interval = 1, friction = 0.99, record_every_x_steps = 1):
    ''' The environment will be a set size and you will have to pass an array of particles in it so that they can
    interact with the environment and with each other'''
    self.time_interval = time_interval
    self.epsilon = 0.00001
    self.total_steps = 0 
    self.size = size
    self.coulomb_constant = 1
    [particle.set_env(self) for particle in particles]
    self.particles = particles
    [particle.set_personal_global_particle_data() for particle in self.particles]
    [particle.set_other_particles() for particle in self.particles]
    self.friction = friction
    self.set_global_particle_data()

    self.record_every_x_steps = record_every_x_steps

    self.update_variables_based_off_time_interval()

    # For logging information
    self.history = {"position": [],
                    "velocity": []}

    # For interpolation later (NEVER USED BTW)
    self.smallest_particle_size = self.particles[0].size
    for particle in self.particles:
      if particle.size < self.smallest_particle_size:
        self.smallest_particle_size = particle.size

  def update_variables_based_off_time_interval(self):
    '''This makes it so that (in theory) particles should act the same no matter the time interval but this isn't true because of float precision and distance that particles move each step and also because attractive forces are calcualted at the step so if you make steps more frequently the particle will get a higher velocity'''
    for particle in self.particles:
      particle.set_friction(particle.friction ** (self.time_interval))
    self.friction = self.friction ** (self.time_interval)

  def step(self):
    '''Runs all the functions needed to make the particles move one step'''
    self.total_steps += 1
    [particle.apply_forces() for particle in self.particles]
    [particle.step() for particle in self.particles]
    self.make_sure_particles_arent_inside_one_another()
    
    # Saves the timestep if the time is wright
    if self.total_steps % self.record_every_x_steps == 0:
      self.save_timestep()
  
  def make_sure_particles_arent_inside_one_another(self):
    '''Makes sure that particles arent inside one another'''
    for particle in self.particles:
      coliding_particles = particle.check_for_particles_coliding()
      if len(coliding_particles) > 0:
        for other in coliding_particles:
          particle.push(other) 

  def save_timestep(self):
    '''Saves position and velocity of particles'''
    self.history["position"].append(self.get_particles_position())
    self.history["velocity"].append(self.get_particles_velocity())

  def get_particles_velocity(self):
    return [particle.get_velocity() for particle in self.particles]

  def get_particles_position(self):
    return [particle.get_position() for particle in self.particles]

  def set_global_particle_data(self):
    self.global_particle_data = [np.array([particle.position for particle in self.particles], dtype = np.float16), np.array([particle.charge for particle in self.particles], dtype = np.float16)]
  
  def all_particles_stats(self):
    string = ""
    for particle in self.particles:
      string += particle.info() + "\n"
    return string

  def run(self, n_steps, verbose = 0, display_stats = False, frames_per_second = 60, clear_output = True):
    '''Runs n amount of steps and has the ability to display stats and the actual environment'''
    sleep_for = (1/frames_per_second)
    for i in range(n_steps):
      self.step()
      if verbose > 0:
        if clear_output:
          IPython.display.clear_output(True)
        print(self.display())
        if frames_per_second > 0:
          time.sleep(sleep_for)
      if display_stats:
        [print(p.info()) for p in self.particles]

  def display(self):
    '''Outputs a string that is printable where we can actually see what is going on in the environment'''
    verticle_wall = "".join(["-" for z in range(self.size[0] + 2)]) + "\n"
    display = [[" " for z in range(self.size[0])] for z in range(self.size[1])]
    for particle in self.particles:
      if (particle.position[0] > 0 and particle.position[1] > 0) and (round(particle.position[0]) < self.size[0] and round(particle.position[1]) < self.size[1]):
        display[round(particle.position[1])][round(particle.position[0])] = particle.icon
      
    display = ["|"+"".join(line) + "|\n" for line in display]
    
    display = verticle_wall + "".join(display) + verticle_wall
    return display

In [ ]:
### Create your particles that will be simulated here
# icon - how it will be displayed
# position - initial position
# velocity - initial velocity
# charge - charge of particle
# friction - how much friction it will have with the environment and when it interacts with other particles (energy lost)
# mass - how much mass the particle has
# size - how big the particle will be

# How big the box where the simulation will take place
size_of_env = (98, 23)

def random_particles(n):
  '''Create n random particles'''
  return [Particle(None, icon = chr(random.randint(33 , 256)), position = [random.random() * size_of_env[0], random.random() * size_of_env[1]], velocity = [(random.random() - 0.5) * 2, (random.random() - 0.5) * 2], charge = (random.random() - 0.5) * 5,size = random.random() + 1, mass = random.random() * 5) for z in range(n)]

### Create the environment here
# particles - the particles that you want int your env
# size - size of the environment
# time_interval - a relative measure of how much time happens between each step, if wacky things happen then decrease time_interval because then smaller steps will happen and hopefully less crazy stuff
# friction - how much energy is lost per step, relative to time_interval
# record_every_x_steps - how often to record the position and velocities of the particles in the simulation

env = Environment(random_particles(10), size = size_of_env,time_interval = .05, friction = 0.9, record_every_x_steps=100)
print("THIS IS HOW THE SIMULATION LOOKS")
print(env.display())

In [ ]:
### Run through the simulation
# n_steps - how many steps to run
# display_stats - whether or not to show information on all of the particles in the environment
# verbose - if its greater than 0 we will display the environment
# frames_per_second - how many instances of the environment to show per second
# clear_output - if True then it will display just one frame at a time, else it will display all frames and not clear 

env.run(n_steps = 1000, verbose = 1, display_stats = True)
